In [30]:
import pandas, copy

from tqdm import tqdm

from collections import defaultdict

from src.utils import build_exact_table, build_bootstrapped_table, create_predictions_table, plot_truthtables

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
who_drugs = list(pandas.read_csv('dat/drugs/who2_drugs.csv').drug)
plate_drugs = list(pandas.read_csv('dat/drugs/plate_drugs.csv').drug)
other_drugs = list(pandas.read_csv('dat/drugs/other_drugs.csv').drug)
mgit_drugs = list(pandas.read_csv('dat/drugs/mgit_drugs.csv').drug)

drug_names_table = pandas.read_csv("dat/drugs/drug_names_lookup.csv")
drug_names_table.set_index("DRUG", inplace=True)
drug_names_lookup = {}
for idx, row in drug_names_table.iterrows():
    drug_names_lookup[idx] = row.DRUG_NAME.capitalize()

In [3]:
PHENOTYPES = pandas.read_csv('dat/PHENOTYPES.csv')
PHENOTYPES.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
PHENOTYPES.sort_index(inplace=True)
PHENOTYPES[:2]

UNIQUEID  \
ENA_RUN_ACCESSION DRUG                                                 
ERR13286038       BDQ   site.10.subj.LA00559418.lab.LA00559418.iso.1   
                  LZD   site.10.subj.LA00559418.lab.LA00559418.iso.1   

                       BINARY_PHENOTYPE PHENOTYPE_QUALITY PHENOTYPE_METHOD  
ENA_RUN_ACCESSION DRUG                                                      
ERR13286038       BDQ                 R               NaN             MGIT  
                  LZD                 S               NaN             MGIT

In [4]:
RAW_PREDICTIONS = pandas.read_csv('dat/RAW_PREDICTIONS.csv')
RAW_PREDICTIONS.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
RAW_PREDICTIONS[:3]

PREDICTION
ENA_RUN_ACCESSION DRUG           
ERR4829376        AMI           S
                  BDQ           S
                  CAP           S

In [5]:
RAW_EFFECTS = pandas.read_csv('dat/RAW_EFFECTS.csv')
RAW_EFFECTS.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
RAW_EFFECTS.sort_index(inplace=True)
print(f"There are {RAW_EFFECTS.IS_NULL.sum()} null calls and {RAW_EFFECTS.IS_MINOR_ALLELE.sum()} minor allele calls in the main effects table")
RAW_EFFECTS[:3]

There are 375 null calls and 1057 minor allele calls in the main effects table


GENE      MUTATION PREDICTION  EPISTASIS  \
ENA_RUN_ACCESSION DRUG                                               
ERR13286038       BDQ   Rv0678  138_ins_g:36          S      False   
                  CAP     tlyA          L11L          S      False   
                  CFZ   Rv0678  138_ins_g:36          S      False   

                        IS_MINOR_ALLELE  IS_NULL  
ENA_RUN_ACCESSION DRUG                            
ERR13286038       BDQ              True    False  
                  CAP             False    False  
                  CFZ              True    False

Most of the null calls are in the *rrs* gene so will affect the aminoglycosides

In [6]:
RAW_EFFECTS[RAW_EFFECTS.IS_NULL].GENE.value_counts()

GENE
rrs       328
rpoB       19
pncA        8
Rv0678      4
embB        4
gid         2
fabG1       2
gyrB        2
tlyA        2
rplC        2
ethA        1
katG        1
Name: count, dtype: int64

Whilst there are over 50 minor allele calls in genes that could affect resistance prediction for the fluorouinolones, rifampicin, bedaquline, pyrazinamide and ethambutol.

In [7]:
RAW_EFFECTS[RAW_EFFECTS.IS_MINOR_ALLELE].GENE.value_counts()[:15]

GENE
gyrA      244
rrs       184
Rv0678    160
rpoB      102
pncA       74
fabG1      54
embB       51
gid        47
gyrB       36
katG       25
eis        20
rpsL       17
ethA       14
rplC       13
rrl         9
Name: count, dtype: int64

Let's now build three EFFECTS tables that progressively include these nuances and from these build three PREDICTIONS tables that we can analyse by joining to PHENOTYPES

In [8]:
EFFECTS={}
EFFECTS[1] = copy.deepcopy(RAW_EFFECTS[~(RAW_EFFECTS.IS_MINOR_ALLELE) & ~(RAW_EFFECTS.IS_NULL)])
EFFECTS[1]['SET'] = 'basic'
EFFECTS[2] = copy.deepcopy(RAW_EFFECTS[~(RAW_EFFECTS.IS_MINOR_ALLELE)])
EFFECTS[2]['SET'] = 'nulls'
EFFECTS[3] = copy.deepcopy(RAW_EFFECTS)
EFFECTS[3]['SET'] = 'nulls+minors'
assert len(EFFECTS[3]) >  len(EFFECTS[2]) >  len(EFFECTS[1])

PREDICTIONS={}
for i in [1,2,3]:
    PREDICTIONS[i] = pandas.DataFrame(create_predictions_table(EFFECTS[i], who_drugs))

assert len(RAW_PREDICTIONS) == len(PREDICTIONS[1]) == len(PREDICTIONS[2]) == len(PREDICTIONS[3])

EFFECTS = pandas.concat(EFFECTS.values())
EFFECTS.reset_index(inplace=True)
EFFECTS.set_index(['SET','ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
EFFECTS.to_csv('dat/EFFECTS.csv')

PREDICTIONS= pandas.concat(PREDICTIONS.values())
PREDICTIONS.to_csv('dat/PREDICTIONS.csv')

100%|██████████| 53398/53398 [00:02<00:00, 23517.03it/s]


These tables each contain three scenarios (i) `basic` which ignores all null and minor alleles and is therefore closest to the WHOv2 catalogue as written, (ii) `nulls` which makes an `F` call at genetic loci associated with resistance where there are insufficient reads (two or fewer) and (iii) `nulls+minors` which adds in calls where there are three or more reads supporting a resistance-associated variant.

In [9]:
EFFECTS[:2]

GENE MUTATION PREDICTION  EPISTASIS  \
SET   ENA_RUN_ACCESSION DRUG                                        
basic ERR13286038       CAP   tlyA     L11L          S      False   
                        DLM   fgd1    K270M          U      False   

                              IS_MINOR_ALLELE  IS_NULL  
SET   ENA_RUN_ACCESSION DRUG                            
basic ERR13286038       CAP             False    False  
                        DLM             False    False

In [10]:
PREDICTIONS[:2]

PREDICTION
SET   ENA_RUN_ACCESSION DRUG           
basic ERR13286038       INH           R
                        RIF           R

Let's check that has all worked, first by picking a sample and looking at its rows in an EFFECTS table

In [11]:
check_sample = 'ERR13286038'
effects = copy.deepcopy(EFFECTS)
effects.reset_index(inplace=True)
effects = effects[(effects.SET=='nulls') & (effects.ENA_RUN_ACCESSION==check_sample)]
effects.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
effects

SET   GENE MUTATION PREDICTION  EPISTASIS  \
ENA_RUN_ACCESSION DRUG                                                
ERR13286038       CAP   nulls   tlyA     L11L          S      False   
                  DLM   nulls   fgd1    K270M          U      False   
                  EMB   nulls   embB    A409P          U      False   
                  EMB   nulls   embB    E405D          U      False   
                  ETH   nulls   ethA    Y211S          U      False   
                  ETH   nulls  fabG1    c-15t          R      False   
                  INH   nulls  fabG1    c-15t          R      False   
                  INH   nulls   katG    S315T          R      False   
                  LEV   nulls   gyrA     A90V          R      False   
                  LEV   nulls   gyrA     D94G          R      False   
                  LEV   nulls   gyrA     E21Q          S      False   
                  LEV   nulls   gyrA    G668D          S      False   
                  LEV   nulls   gyrA     S95T          S      False   
                  MXF   nulls   gyrA     A90V          R      False   
                  MXF   nulls   gyrA     D94G          R      False   
                  MXF   nulls   gyrA     E21Q          S      False   
                  MXF   nulls   gyrA    G668D          S      False   
                  MXF   nulls   gyrA     S95T          S      False   
                  PZA   nulls   pncA     H71Y          R      False   
                  RIF   nulls   rpoB    D103D          S      False   
                  RIF   nulls   rpoB    H445Y          R      False   
                  STM   nulls    gid    A134E          R      False   

                        IS_MINOR_ALLELE  IS_NULL  
ENA_RUN_ACCESSION DRUG                            
ERR13286038       CAP             False    False  
                  DLM             False    False  
                  EMB             False    False  
                  EMB             False    False  
                  ETH             False    False  
                  ETH             False    False  
                  INH             False    False  
                  INH             False    False  
                  LEV             False    False  
                  LEV             False    False  
                  LEV             False    False  
                  LEV             False    False  
                  LEV             False    False  
                  MXF             False    False  
                  MXF             False    False  
                  MXF             False    False  
                  MXF             False    False  
                  MXF             False    False  
                  PZA             False    False  
                  RIF             False    False  
                  RIF             False    False  
                  STM             False    False

Now compare that to the calculated per drug predictions

In [12]:
create_predictions_table(effects, who_drugs)

  0%|          | 0/22 [00:00<?, ?it/s]

100%|██████████| 22/22 [00:00<00:00, 21096.18it/s]


PREDICTION
SET   ENA_RUN_ACCESSION DRUG           
nulls ERR13286038       INH           R
                        RIF           R
                        PZA           R
                        EMB           U
                        BDQ           S
                        LZD           S
                        MXF           R
                        LEV           R
                        CFZ           S
                        DLM           U
                        AMI           S
                        STM           R
                        ETH           R
                        KAN           S
                        CAP           S

Since we have a `RAW_PREDICTIONS` table parsed directly from the individual sample `json` files we can compare `PREDICTIONS['nulls+minors']` to that and they should be identical.

A direct way of doing that is to make an antibiogram for each sample (which will automatically be in alphabetical order of drug), and then look for samples where they are different.

In [13]:
def make_antibiogram(row):
    antibiogram=''
    for i in who_drugs:
        antibiogram += row[('PREDICTION',i)]
    return antibiogram

df1 = RAW_PREDICTIONS.unstack()
df1['ANTIBIOGRAM'] = df1.apply(make_antibiogram, axis=1)

PREDICTIONS.reset_index(inplace=True)
df2 = PREDICTIONS[PREDICTIONS.SET=='nulls+minors']
PREDICTIONS.set_index(['SET','ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
df2.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
df2=df2[['PREDICTION']].unstack()
# df2=df2.droplevel(0, axis=1)
df2['ANTIBIOGRAM'] = df2.apply(make_antibiogram, axis=1)

mask = (df1.ANTIBIOGRAM != df2.ANTIBIOGRAM)
assert len(df1[mask]) == 0, 'some samples have different antibiograms!'

Having the (alphabetical) antibiogram is useful, so let's calculate it for all three scenarios

In [14]:
df = PREDICTIONS.unstack()
df['ANTIBIOGRAM'] = df.apply(make_antibiogram, axis=1)
df = df[[('ANTIBIOGRAM','')]]
df = df.droplevel(1, axis=1)
df.reset_index(inplace=True)
df1 = df[df.SET=='basic']
df2 = df[df.SET=='nulls']
df3 = df[df.SET=='nulls+minors']
df1.set_index('ENA_RUN_ACCESSION', inplace=True)
df2.set_index('ENA_RUN_ACCESSION', inplace=True)
df3.set_index('ENA_RUN_ACCESSION', inplace=True)
df1 = df1.rename(columns={'SET':"SET1",'ANTIBIOGRAM':'ANTIBIOGRAM1'})
df2 = df2.rename(columns={'SET':"SET2",'ANTIBIOGRAM':'ANTIBIOGRAM2'})
df3 = df3.rename(columns={'SET':"SET3",'ANTIBIOGRAM':'ANTIBIOGRAM3'})
ANTIBIOGRAMS = df1.join(df2).join(df3)
ANTIBIOGRAMS.to_csv('dat/ANTIBIOGRAMS.csv')
ANTIBIOGRAMS[:3]

,SET1,ANTIBIOGRAM1,SET2,ANTIBIOGRAM2,SET3,ANTIBIOGRAM3
ENA_RUN_ACCESSION,,,,,,
ERR13286038,basic,RRRUSSRRSUSRRSS,nulls,RRRUSSRRSUSRRSS,nulls+minors,RRRUSSRRSUSRRSS
ERR13286039,basic,RSSSSSSSSSSSSSS,nulls,RSSSSSSSSSSSSSS,nulls+minors,RSSSRSSSRSSSSSS
ERR13286042,basic,RRRRSURRSSRRRRR,nulls,RRRRSURRSSRRRRR,nulls+minors,RRRRSURRSSRRRRR


We can now look to see how many samples out of 2,663 change between scenarios

In [15]:
print(f"Calling nulls at resistance loci changes the antibiogram of {(ANTIBIOGRAMS.ANTIBIOGRAM1!=ANTIBIOGRAMS.ANTIBIOGRAM2).sum()} samples,\
whilst also calling minor alleles leads to {(ANTIBIOGRAMS.ANTIBIOGRAM2!=ANTIBIOGRAMS.ANTIBIOGRAM3).sum()} samples changing antibiogram")

Calling nulls at resistance loci changes the antibiogram of 33 samples,whilst also calling minor alleles leads to 210 samples changing antibiogram


In [16]:
results_drug_order = pandas.read_csv('dat/drugs/results_drugs.csv')
results_drug_order.set_index(['pDST method', 'drug'], inplace=True)
list(results_drug_order.index)

[('UKMYC', 'INH'),
 ('UKMYC', 'RIF'),
 ('MGIT', 'PZA'),
 ('UKMYC', 'EMB'),
 ('MGIT', 'BDQ'),
 ('UKMYC', 'LZD'),
 ('UKMYC', 'MXF'),
 ('UKMYC', 'LEV'),
 ('UKMYC', 'CFZ'),
 ('UKMYC', 'DLM'),
 ('UKMYC', 'AMI'),
 ('MGIT', 'STM'),
 ('UKMYC', 'ETH'),
 ('UKMYC', 'KAN'),
 ('MGIT', 'CAP')]

In [17]:
results={}
PREDICTIONS.reset_index(inplace=True)
PREDICTIONS.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)

for i in tqdm(['basic','nulls','nulls+minors']):
    
    predictions = PREDICTIONS[PREDICTIONS.SET==i]
    df = copy.deepcopy(predictions.join(PHENOTYPES, how='inner'))

    plate_df = df[df.PHENOTYPE_METHOD=='UKMYC']
    plate_df.reset_index(inplace=True)
    table = []
    table = build_exact_table(table, plate_df, plate_drugs, ['HIGH', 'ALL'], i, 'UKMYC', include_fails=False)
    table = build_bootstrapped_table(table,plate_df, plate_drugs, ['HIGH', 'ALL'], i, 'UKMYC', include_fails=False)

    mgit_df = df[df.PHENOTYPE_METHOD=='MGIT']
    mgit_df.reset_index(inplace=True)
    table = build_exact_table(table, mgit_df, mgit_drugs, ['ALL'], i, 'MGIT', include_fails=False)
    table = build_bootstrapped_table(table,mgit_df, mgit_drugs, ['ALL'], i, 'MGIT', include_fails=False)
    results[i] = pandas.DataFrame(table, columns=['set','drug','method','quality','dataset','sensitivity','sensitivity_sem','specificity','specificity_sem','PPV','PPV_sem', 'RR', 'SR', 'UR', 'RS', 'SS', 'US', 'Total'])

    results[i].set_index(['method','drug'],inplace=True)
    results[i] = results[i][results[i].index.isin(list(results_drug_order.index))]
    results[i].reset_index(inplace=True)
    results[i].drug = results[i].drug.astype('category')
    results[i].drug = results[i].drug.cat.set_categories(who_drugs)
    results[i].sort_values(['drug', 'dataset', 'quality'], inplace=True)
    results[i].set_index(['set','drug','method','dataset', 'quality'],inplace=True)

results = pandas.concat(results.values())
results.to_csv('dat/RESULTS.csv')
results


100%|██████████| 3/3 [00:32<00:00, 10.86s/it]


sensitivity  sensitivity_sem  \
set          drug method dataset        quality                                 
basic        INH  UKMYC  bootstrapped50 ALL        91.240875         0.480167   
                                        HIGH       93.746796         0.336085   
                         entire         ALL        91.200000         0.000000   
                                        HIGH       93.913043         0.000000   
             RIF  UKMYC  bootstrapped50 ALL        93.729501         0.406094   
...                                                      ...              ...   
nulls+minors KAN  UKMYC  bootstrapped50 HIGH       84.298262         0.713334   
                         entire         ALL        76.470588         0.000000   
                                        HIGH       84.615385         0.000000   
             CAP  MGIT   bootstrapped50 ALL        74.983621         0.941786   
                         entire         ALL        75.098814         0.000000   

                                                 specificity  specificity_sem  \
set          drug method dataset        quality                                 
basic        INH  UKMYC  bootstrapped50 ALL        95.395150         0.367757   
                                        HIGH       96.624866         0.352979   
                         entire         ALL        95.600000         0.000000   
                                        HIGH       96.412556         0.000000   
             RIF  UKMYC  bootstrapped50 ALL        95.957331         0.314197   
...                                                      ...              ...   
nulls+minors KAN  UKMYC  bootstrapped50 HIGH       98.322163         0.199675   
                         entire         ALL        98.730606         0.000000   
                                        HIGH       98.543689         0.000000   
             CAP  MGIT   bootstrapped50 ALL        98.295397         0.200364   
                         entire         ALL        98.254799         0.000000   

                                                       PPV   PPV_sem     RR  \
set          drug method dataset        quality                               
basic        INH  UKMYC  bootstrapped50 ALL      95.206597  0.370676    NaN   
                                        HIGH     96.618858  0.342746    NaN   
                         entire         ALL      95.397490  0.000000  456.0   
                                        HIGH     96.428571  0.000000  432.0   
             RIF  UKMYC  bootstrapped50 ALL      95.445435  0.375315    NaN   
...                                                    ...       ...    ...   
nulls+minors KAN  UKMYC  bootstrapped50 HIGH     95.369505  0.574964    NaN   
                         entire         ALL      96.086957  0.000000  221.0   
                                        HIGH     96.069869  0.000000  220.0   
             CAP  MGIT   bootstrapped50 ALL      94.980289  0.593964    NaN   
                         entire         ALL      95.000000  0.000000  190.0   

                                                   SR    UR    RS     SS  \
set          drug method dataset        quality                            
basic        INH  UKMYC  bootstrapped50 ALL       NaN   NaN   NaN    NaN   
                                        HIGH      NaN   NaN   NaN    NaN   
                         entire         ALL      33.0  11.0  22.0  448.0   
                                        HIGH     23.0   5.0  16.0  404.0   
             RIF  UKMYC  bootstrapped50 ALL       NaN   NaN   NaN    NaN   
...                                               ...   ...   ...    ...   
nulls+minors KAN  UKMYC  bootstrapped50 HIGH      NaN   NaN   NaN    NaN   
                         entire         ALL      56.0  12.0   9.0  650.0   
                                        HIGH     34.0   6.0   9.0  563.0   
             CAP  MGIT   bootstrapped50 ALL       NaN   NaN   NaN    NaN   
 

In [18]:
results.reset_index(inplace=True)
df = copy.deepcopy(results[results.set=='nulls+minors'])
df = df[(df.dataset=='bootstrapped50') & (df.quality=='ALL')]
df = df[['method', 'drug', 'sensitivity',
       'sensitivity_sem', 'specificity', 'specificity_sem', 'PPV', 'PPV_sem']]

for col in ['sensitivity', 'sensitivity_sem', 'specificity', 'specificity_sem', 'PPV', 'PPV_sem']:
    df[col] = df[col].map('{:,.1f}'.format)
for col in ['sensitivity', 'specificity', 'PPV']:
    df[col] = df[col] + ' ±' + df[col+'_sem']
df = df[['method', 'drug', 'sensitivity',
        'specificity', 'PPV', ]]
df.set_index(['method', 'drug'], inplace=True)
df

sensitivity specificity        PPV
method drug                                   
UKMYC  INH    92.5 ±0.5   94.6 ±0.4  94.5 ±0.4
       RIF    95.3 ±0.4   94.9 ±0.3  94.5 ±0.4
MGIT   PZA    85.1 ±0.7   96.7 ±0.4  95.7 ±0.5
UKMYC  EMB    86.7 ±0.8   84.1 ±0.5  68.4 ±1.0
MGIT   BDQ    46.1 ±1.0   98.6 ±0.2  97.2 ±0.4
UKMYC  LZD    28.1 ±1.9   99.8 ±0.1  94.3 ±1.9
       MXF    85.8 ±0.8   93.8 ±0.4  84.9 ±0.9
       LEV    82.3 ±0.7   96.4 ±0.3  91.7 ±0.7
       CFZ     8.1 ±0.6   98.0 ±0.2  62.4 ±3.1
       DLM    12.3 ±1.1   99.9 ±0.0  95.5 ±1.8
       AMI    74.3 ±1.0   99.4 ±0.1  98.1 ±0.3
MGIT   STM    80.7 ±0.8   95.3 ±0.4  92.3 ±0.6
UKMYC  ETH    72.2 ±1.0   87.0 ±0.5  69.4 ±1.0
       KAN    76.7 ±1.0   98.7 ±0.2  96.0 ±0.6
MGIT   CAP    75.0 ±0.9   98.3 ±0.2  95.0 ±0.6

In [19]:
print(df.to_latex(column_format='llrrr', multirow=True))

\begin{tabular}{llrrr}
\toprule
 &  & sensitivity & specificity & PPV \\
method & drug &  &  &  \\
\midrule
\multirow[t]{2}{*}{UKMYC} & INH & 92.5 ±0.5 & 94.6 ±0.4 & 94.5 ±0.4 \\
 & RIF & 95.3 ±0.4 & 94.9 ±0.3 & 94.5 ±0.4 \\
\cline{1-5}
MGIT & PZA & 85.1 ±0.7 & 96.7 ±0.4 & 95.7 ±0.5 \\
\cline{1-5}
UKMYC & EMB & 86.7 ±0.8 & 84.1 ±0.5 & 68.4 ±1.0 \\
\cline{1-5}
MGIT & BDQ & 46.1 ±1.0 & 98.6 ±0.2 & 97.2 ±0.4 \\
\cline{1-5}
\multirow[t]{6}{*}{UKMYC} & LZD & 28.1 ±1.9 & 99.8 ±0.1 & 94.3 ±1.9 \\
 & MXF & 85.8 ±0.8 & 93.8 ±0.4 & 84.9 ±0.9 \\
 & LEV & 82.3 ±0.7 & 96.4 ±0.3 & 91.7 ±0.7 \\
 & CFZ & 8.1 ±0.6 & 98.0 ±0.2 & 62.4 ±3.1 \\
 & DLM & 12.3 ±1.1 & 99.9 ±0.0 & 95.5 ±1.8 \\
 & AMI & 74.3 ±1.0 & 99.4 ±0.1 & 98.1 ±0.3 \\
\cline{1-5}
MGIT & STM & 80.7 ±0.8 & 95.3 ±0.4 & 92.3 ±0.6 \\
\cline{1-5}
\multirow[t]{2}{*}{UKMYC} & ETH & 72.2 ±1.0 & 87.0 ±0.5 & 69.4 ±1.0 \\
 & KAN & 76.7 ±1.0 & 98.7 ±0.2 & 96.0 ±0.6 \\
\cline{1-5}
MGIT & CAP & 75.0 ±0.9 & 98.3 ±0.2 & 95.0 ±0.6 \\
\cline{1-5}
\bottomrule

In [40]:
for set in ['basic', 'nulls', 'nulls+minors']:

    df = results[(results.set==set)]

    plot_truthtables(df, ['ALL','HIGH'], f'table-{set}-')


In [20]:
PHENOTYPES[:3]

UNIQUEID  \
ENA_RUN_ACCESSION DRUG                                                 
ERR13286038       BDQ   site.10.subj.LA00559418.lab.LA00559418.iso.1   
                  LZD   site.10.subj.LA00559418.lab.LA00559418.iso.1   
ERR13286039       BDQ   site.10.subj.JE01389322.lab.JE01389322.iso.1   

                       BINARY_PHENOTYPE PHENOTYPE_QUALITY PHENOTYPE_METHOD  
ENA_RUN_ACCESSION DRUG                                                      
ERR13286038       BDQ                 R               NaN             MGIT  
                  LZD                 S               NaN             MGIT  
ERR13286039       BDQ                 R               NaN             MGIT

In [21]:
PREDICTIONS[:3]

SET PREDICTION
ENA_RUN_ACCESSION DRUG                  
ERR13286038       INH   basic          R
                  RIF   basic          R
                  PZA   basic          R

In [22]:
UKMYC = pandas.read_csv('dat/UKMYC_1000_samples.csv')
UKMYC[:3]

,ENA_RUN_ACCESSION,UNIQUEID,ENA_STUDY_ACCESSION,ENA_SAMPLE_ACCESSION,HAS_ALL_DRUGS,N_SUSCEPTIBLE,N_RESISTANT,N_HIGH_QUALITY,AMI,BDQ,...,EMB_QUALITY,ETH_QUALITY,INH_QUALITY,KAN_QUALITY,LEV_QUALITY,LZD_QUALITY,MXF_QUALITY,RIF_QUALITY,FASTQ_FTP,POS_AVG_GROWTH
0,ERR4810791,site.02.subj.0068.lab.22A018.iso.1,PRJEB41194,SAMEA7542366,True,6,6,9,S,S,...,HIGH,HIGH,HIGH,HIGH,LOW,LOW,HIGH,HIGH,ftp.sra.ebi.ac.uk/vol1/fastq/ERR481/001/ERR481...,44.68
1,ERR4810943,site.02.subj.0091.lab.22A043.iso.1,PRJEB41194,SAMEA7544003,True,5,7,11,S,S,...,HIGH,HIGH,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,ftp.sra.ebi.ac.uk/vol1/fastq/ERR481/003/ERR481...,43.23
2,ERR4810948,site.02.subj.0096.lab.22A048.iso.1,PRJEB41194,SAMEA7544008,True,5,7,11,S,R,...,HIGH,HIGH,HIGH,HIGH,HIGH,HIGH,HIGH,HIGH,ftp.sra.ebi.ac.uk/vol1/fastq/ERR481/008/ERR481...,26.20
